In [ ]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.serializers import CSVSerializer
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-fernandosousa'
prefix = 'perceptron/op_logicos'

instance_type = 'local' # trocar para um instância, por exemplo ml.m5.large, para treinar e implantar no sagemaker

role = sagemaker.get_execution_role()

In [ ]:
# criar dados de treinamento
dados = [
      [0, 0, 0],
      [0, 1, 0],
      [1, 0, 0],
      [1, 1, 1]
    ]
x = np.array(dados)
# save to csv file
np.savetxt('data/train.csv', x, delimiter=',')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

# definir caminhos para sagemaker copiar os dados
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [ ]:
estimator = PyTorch(entry_point='perceptron.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'error':0.1
                    })

In [ ]:
estimator.fit({"training": train_input, "validation": validation_input})

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type, serializer=CSVSerializer())

In [ ]:
response = predictor.predict('1,1,0')
responses

Utilizando instâncias do sagemaker


In [ ]:
instance_type="ml.m5.large"

Exemplo de cobrança

In [ ]:
import pandas as pd

prefix = 'perceptron/cobranca'

dados = pd.read_csv("s3://{}/{}".format(bucket, "Case_cobranca.csv"))




In [ ]:
# preparacao

dados['CLIENTE_NOVO']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'NOVO' else 0)
dados['CLIENTE_INVESTIDOR']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'INVESTIDOR' else 0)    
dados['EMPRESTIMO_CDC']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'CDC' else 0)
dados['EMPRESTIMO_PESSOAL']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'PESSOAL' else 0)
dados['SEXO_M']   = dados['CD_SEXO'].apply(lambda x: 1 if x == 'M' else 0)
dados['IDADE_NORM'] = dados['IDADE'].apply(lambda x: 18 if np.isnan(x) or x < 18 else x) # mínimo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: 76 if x > 76 else x) # máximo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: (x-18)/(76-18)) # normalização entre 0 e 1
dados['QTD_DIVIDAS_NORM'] = dados['QTD_DIVIDAS'].apply(lambda x: 0. if np.isnan(x) else x/16) # normalização entre 0 e 1
dados['ALVO']   = dados['TEMP_RECUPERACAO'].apply(lambda x: 1 if x <= 90 else 0)
dados = dados.drop(['COD', 'TIPO_CLIENTE', 'TIPO_EMPRESTIMO', 'CD_SEXO', 'IDADE', 'QTD_DIVIDAS', 'TEMP_RECUPERACAO'], axis=1)


In [ ]:
#save to csv file
np.savetxt('data/train.csv', dados.to_numpy(), delimiter=',')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

In [ ]:
# definir caminhos para sagemaker copiar os dados
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [ ]:
estimator2 = PyTorch(entry_point='perceptron.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'error':0.3
                    })

In [ ]:
estimator2.fit({"training": train_input, "validation": validation_input})

In [ ]:
predictor2 = estimator2.deploy(initial_instance_count=1, instance_type=instance_type, serializer=CSVSerializer())

In [ ]:
'1,'+','.join(map(str,dados.to_numpy()[0, :-1].tolist()))

In [ ]:
response = predictor2.predict('1,1,0')
response